# X-ray Inception-V3

## Prepare dataset 

In [1]:
import pandas as pd
import numpy as np
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.preprocessing.image import ImageDataGenerator
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(r'/home/gxt/study/chestX-ray/3-10New/dataset/X-ray/Train-Dataset/train', 
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   shuffle = True)
val_generator = val_datagen.flow_from_directory(r'/home/gxt/study/chestX-ray/3-10New/dataset/X-ray/Train-Dataset/val', 
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

test_generator = test_datagen.flow_from_directory(r'/home/gxt/study/chestX-ray/3-10New/dataset/X-ray/Train-Dataset/test', 
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   shuffle = False)

## Prepare Models 

In [2]:
#from tensorflow.keras.applications.inception_v3 import InceptionV3
# import the necessary packages
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, models, layers
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [46]:
#Add on a couple of custom CNN layers on top of the Inception V3 model. 
baseModel = InceptionV3(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# Compose the final model
model = Model(inputs=baseModel.input, outputs=headModel)
#for layer in baseModel.layers:
    #layer.trainable = False

In [ ]:
model.summary()

In [48]:
INIT_LR = 1e-5    # This value is specific to what model is chosen: Inception, VGG or ResNet etc.
EPOCHS = 10
#BS = 8

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=2)

print("Compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the full model, since we unfroze the pre-trained weights above
print("Training the full stack model...")
H = model.fit_generator(generator = train_generator, steps_per_epoch=100,validation_data = val_generator, epochs = EPOCHS)

## Performance 

In [ ]:
Testresults = model.evaluate(test_generator)
print("test loss, test acc:", Testresults)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
target_names=[]
print(test_generator.classes);
for key in test_generator.class_indices:
    target_names.append(key)
#print(target_names)    
Y_pred = model.predict(test_generator)
#print(Y_pred)
y_pred = np.argmax(Y_pred,axis=1)
print(y_pred)
#print(confusion_matrix(val_generator.classes, y_pred))
#print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

## Visualize Model 

In [10]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from tensorflow.keras.utils import to_categorical

In [ ]:
# show the confusion matrix, accuracy, sensitivity, and specificity


# plot the training loss and accuracy
N = EPOCHS 
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
#plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
#plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

## Save Model 

In [33]:
model.save('/home/gxt/study/chestX-ray/3-10New/models/Inception-V3.h5')

In [1]:
from tensorflow.keras.models import Model, load_model
model = load_model('/home/gxt/study/chestX-ray/3-10New/models/Inception-V3.h5')

In [ ]:
model.summary()

## Heat Map

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
IMAGE_PATH = r'/home/gxt/study/chestX-ray/3-10New/dataset/X-ray/Train-Dataset/val/normal/IM-0149-0001.jpeg'
LAYER_NAME = 'conv2d_98'
CLASS_INDEX = 1
img = tensorflow.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
img = tensorflow.keras.preprocessing.image.img_to_array(img)

# Load initial model
model = model

# Create a graph that outputs target convolution and output
grad_model = tensorflow.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

# Get the score for target class
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, CLASS_INDEX]

# Extract filters and gradients
output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]
#print(grads)
# Average gradients spatially
weights = tf.reduce_mean(grads, axis=(0, 1))

# Build a ponderated map of filters according to gradients importance
cam = np.ones(output.shape[0:2], dtype=np.float32)

for index, w in enumerate(weights):
    cam += w * output[:, :, index]

# Heatmap visualization
cam = cv2.resize(cam.numpy(), (224, 224))
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())
print(heatmap)
cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)
plt.imshow(output_image, cmap='rainbow')